In [1]:
import os
import pprint
import pandas as pd
import numpy as np
import copy
import glob
from evo.tools import log
from evo.core import metrics
from evo.tools import file_interface
from evo.core import sync
log.configure_logging()
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False
# magic plot configuration
import matplotlib.pyplot as plt

In [2]:
def metrics_dataframe(reference,estimate,algorithm,dataset,trajectory,correct_scale=True,correct_only_scale=False,n=-1,use_aligned_trajectories = True,t_offset = .1):
    reference, estimate = sync.associate_trajectories(reference, estimate)
    est = copy.deepcopy(estimate)
    if use_aligned_trajectories:
        est.align(reference, correct_scale=correct_scale, correct_only_scale=correct_only_scale, n=n)

    if use_aligned_trajectories:
        data = (reference, est) 
    else:
        data = (reference, estimate)
    
    result_dict = {}
    #APE_t
    pose_relation = metrics.PoseRelation.translation_part
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)
    ape_stats = ape_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'ape_t'):ape_stats})
    #APE_rad
    pose_relation = metrics.PoseRelation.rotation_angle_rad
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)
    ape_stats = ape_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'ape_rad'):ape_stats})
    
    #RPE_t
    pose_relation = metrics.PoseRelation.translation_part
    rpe_metric = metrics.RPE(pose_relation)
    rpe_metric.process_data(data)
    rpe_stats = rpe_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'rpe_t'):rpe_stats})
    #RPE_rad
    pose_relation = metrics.PoseRelation.rotation_angle_rad
    rpe_metric = metrics.RPE(pose_relation)
    rpe_metric.process_data(data)
    rpe_stats = rpe_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'rpe_rad'):rpe_stats})
    
    result_df = pd.DataFrame(result_dict)
    return result_df




In [3]:
experiment = "icra23"
dataset = "KITTI"
trajectories = ["00", "01", "02", "03", "04", "05", "06", "07","08", "09", "10"]
algorithms = ["deepvo", "deepvo_se3", "deepvo_quat", "deepvo_quat_geodesic"]

result_df = pd.DataFrame() 
results_dict = {"algorithm": algorithms,
                "dataset": ["KITTI"],
                "MIMIR":trajectories}

for trajectory in trajectories:
    abs_dataset_path = os.path.join(os.getcwd(),experiment)# Edit this to path of dataset w.r.t. current file

    ref_file = os.path.join(abs_dataset_path,"evo_"+trajectory+"_target_absolute.txt")
    timestamp_file =os.getcwd()+"/groundtruths/kitti/data_odometry_calib/dataset/sequences/"+trajectory+"/times.txt"
    ref = file_interface.read_kitti_poses_file(ref_file)

    for algorithm in algorithms:
        est_file = os.path.join(abs_dataset_path,"evo_"+trajectory+"_"+algorithm+"_absolute.txt")
        estimate = file_interface.read_kitti_poses_file(est_file)
        print(ref)
        print(estimate)
        traj_ref, traj_est = sync.associate_trajectories(ref, estimate)
        # df = metrics_dataframe(ref,estimate,algorithm, 'KITTI', trajectory,correct_scale=False,correct_only_scale=False,n=-1,use_aligned_trajectories = False,t_offset = .1)
        # result_df = pd.concat([result_df, df], axis="columns", join="outer")

4539 poses, 3722.186m path length
4539 poses, 7312.976m path length


SyncException: trajectories must be PoseTrajectory3D objects